# Predict Helix Capping Residues #

The goal is to identify residues just before an alpha helix begins or the residues just after the helix ends. This will improve secondary structure predictors becuase they often extend too far or do not start at the right place. 

The CapsDB has annoted sequences of structures of helix capping residues that can be used to train a deep nueral net. We will use a Bidirectional LSTM using phi/psi features to see if it will those will be good predictors.

## 1. Download data ##

## 2. Generate Features ##
### MMTF Pyspark Imports ###

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.webfilters import Pisces
from mmtfPyspark.filters import ContainsLProteinChain
from mmtfPyspark.mappers import StructureToPolymerChains
from mmtfPyspark.ml import ProteinSequenceEncoder
import numpy as np
import pandas as pd
import math

### Custom imports ###

In [2]:
import secondaryStructureExtractorFull
#import mmtfToASA
import os
os.getcwd()

'/home/ec2-user/SageMaker/ProteinFragmenter'

### Configure Spark Context ###

In [ ]:
spark = SparkSession.builder.master("local[8]").appName("DeepCap").getOrCreate()

### Filter out chains not in CapsDB ###

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import concat, col, lit, array_contains

sqlContext = SQLContext(spark)
capsdb = sqlContext.read.parquet('caps_descriptors.parquet')
capsdb_pdbs = capsdb.select(concat(upper(col("pdbId")), lit("."), col("chain")).alias("id")).drop_duplicates()


In [ ]:
capsdb_pdbs.count()

In [ ]:
#from pyspark.sql.types import BooleanType

# def hasHelixCapInfo(enc_obj, ids):
#     return(enc_obj[0] in ids)

# def udf_hasHelixCapInfo(ids):
#     return udf(lambda t: hasHelixCapInfo(t, ids))
#     # I'm not sure how to call these successfully - Sean

capsdb_ids = set()
[capsdb_ids.add(list[0]) for list in capsdb_pdbs.select("id").collect()]

class HasHelixCapInfo(object):
    '''This filter returns true if the structure is in CAPSDB'''
    def __call__(self, t):
        return(t[0] in capsdb_ids)
    # This works but I don't like that it refers to a global variable: not sure how to change that right now - Sean


### Read MMTF File and get a set of L-protein chains ###

In [ ]:
if not os.path.isdir("full"):
    !wget https://mmtf.rcsb.org/v1.0/hadoopfiles/full.tar && tar -xvf full.tar
        
if not os.path.isdir("reduced"):
    !wget https://mmtf.rcsb.org/v1.0/hadoopfiles/reduced.tar && tar -xvf reduced.tar

In [ ]:
pdb = mmtfReader.read_sequence_file('full') \
    .flatMap(StructureToPolymerChains()) \
    .filter(ContainsLProteinChain()) \
    .filter(HasHelixCapInfo())
#pdb.count()
#pdb.take(2)
#?pdb

### Get Torsion angle and secondary structure info ###

In [ ]:
#from mmtfPyspark.datasets import secondaryStructureExtractor

data = secondaryStructureExtractorFull.get_dataset(pdb).toPandas()
#data = secondaryStructureExtractor.get_dataset(pdb).toPandas()

In [ ]:
# Read output of above get_dataset operation from parquet file
parquetPath = '/home/ec2-user/SageMaker/ProteinFragmenter/datacaps.parquet'
dataframe = sqlContext.read.parquet(parquetPath)
data = dataframe.toPandas()
data = data.drop('__index_level_0__', axis=1)

In [ ]:
data.head(10)

In [ ]:
df1 = capsdb.toPandas()
df = pd.merge(data, df1, left_on=('pdbId','chain'), right_on=('pdbid','chain'), how='inner')
df = df[['pdbId', 'chain', 'resi', 'resn', 'phi', 'psi', 'startcap', 'endcap']]


### Create labels

In [ ]:
df['is_cap'] = df.apply(lambda x: 1 if (x['resi'] >= x['startcap'] and x['resi'] <= x['endcap']) else 0, axis=1)
df_caps = df.groupby(["pdbId", "chain", "resi"])['is_cap'].max().reset_index()

In [ ]:
data_caps = pd.merge(data, df_caps, left_on=('pdbId','chain', 'resi'), right_on=('pdbId','chain', 'resi'), how='inner')

In [ ]:
from Bio.PDB.Polypeptide import aa3
one_hot_encoded = pd.DataFrame(data_caps.resn.apply(lambda x: secondaryStructureExtractorFull.get_residue(x)).tolist(), columns=aa3)
one_hot_encoded.head()
data_caps = data_caps.join(one_hot_encoded)
data_caps.head()

In [ ]:
data_caps.head()

In [ ]:
#test = training_data[0:128,:,:]

def is_cap(pdbId, chain, resi, is_cap):
    if is_cap == 1:
        return([1,0])
    elif is_cap == 0:
        return([0,1])
    else:
        raise ValueError("is_cap must be 0 or 1")

def angle_to_cos(angle):
    if(angle == 0 or np.isnan(angle)):
        return 0
    else:
        #print("working")
        return np.cos(np.pi * angle/180)

def angle_to_sin(angle):
    if(angle == 0 or np.isnan(angle)):
        return 0
    else:
        #print("working")
        return np.sin(np.pi * angle/180)


In [ ]:
groups = data_caps.groupby(["pdbId", "chain"])
                           # num pdbs,    max len of seqs, num features
training_data = np.zeros((groups.ngroups, 5000, 24), dtype=float)
                           # num pdbs,    max len of seqs, length of one-hot encoded target
truth = np.zeros((groups.ngroups, 5000, 2), dtype=int)
truth_lagged = np.zeros((groups.ngroups, 5000, 2), dtype=int)


In [ ]:
for i, ((pdbid, chain), group) in enumerate(groups):
    for j, featuretuple in enumerate(group.itertuples()):
        if j>=5000: break
        truth[i,j,:] = is_cap(featuretuple.pdbId, featuretuple.chain, featuretuple.resi, featuretuple.is_cap)
        training_data[i,j,:] = (angle_to_cos(featuretuple.phi), angle_to_sin(featuretuple.phi), 
                              angle_to_cos(featuretuple.psi), angle_to_sin(featuretuple.psi), featuretuple.ALA,
                              featuretuple.CYS,featuretuple.ASP,featuretuple.GLU,featuretuple.PHE,
                              featuretuple.GLY,featuretuple.HIS,featuretuple.ILE,featuretuple.LYS,
                              featuretuple.LEU,featuretuple.MET,featuretuple.ASN,featuretuple.PRO,
                              featuretuple.GLN,featuretuple.ARG,featuretuple.SER,featuretuple.THR,
                              featuretuple.VAL,featuretuple.TRP,featuretuple.TYR)
        if (j > 0):
            truth_lagged[i,j-1,:] = truth[i,j,:]
        

In [ ]:
training_data[10,2,:]
#truth[2]
#truth[8,:,0].sum()
#truth.shape
training_data.shape
#os.getcwd()
training_data = training_data[:, 0:100, :]
truth = truth[:, 0:100, :]
truth_lagged = truth_lagged[:, 0:100, :]

In [ ]:
# get subset of training data, if needed for faster debugging

#training_data = HDF5Matrix('features.h5', 'training_data')
import h5py
hf = h5py.File('full_dataset_features/features.h5', 'r')
training_data = hf.get('training_data').value
hf.close()

hf = h5py.File('full_dataset_features/truth.h5', 'r')
truth = hf.get('truth').value
hf.close()

hf = h5py.File('full_dataset_features/truthlag.h5', 'r')
truth_lagged = hf.get('truthlag').value
hf.close()

### Write features, labels to H5 file ###

In [ ]:
import h5py
os.remove('features.h5')
os.remove('truth.h5')
os.remove('truthlag.h5')

h5f = h5py.File('features.h5', 'w')
h5f.create_dataset('training_data', data=training_data)
h5f.close()

h5tr = h5py.File('truth.h5', 'w')
h5tr.create_dataset('truth', data=truth)
h5tr.close()

h5trl = h5py.File('truthlag.h5', 'w')
h5trl.create_dataset('truthlag', data=truth_lagged)
h5trl.close()

#os.remove("features.h5")

### Terminate Spark ###

In [ ]:
spark.stop()

## 4. Build Bidirectional LSTM ##

In [30]:
def create_and_train_model(num_features, num_outputs=2, latent_dim=100):
    """Create a Seq2Seq Bidirectional LSTM
    From: https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
    
    Parameters
    ----------
    num_features : int
        The number of features in your trianing data
    num_outputs : int
        Number of outputs to rpedict, i.e. number of classes or 2 for binary
        
    Returns
    -------
    A new Keras Seq2Seq Bidirectional LSTM
    """
    
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(None, num_features), name="input")
    #mask = Masking(mask_value=0.)(encoder_inputs) # Masking is not supported by CuDNNLSTM
    encoder = CuDNNLSTM(latent_dim, return_state=True, name="encoder_LSTM") #, activation='relu')
    #keras.layers.add([encoder, bn])
    
    #encoder = Bidirectional(LSTM(latent_dim, return_state=True))
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    #bn = BatchNormalization()(encoder_outputs)
    #encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
    #state_h = Concatenate()([forward_h, backward_h])
    #state_c = Concatenate()([forward_c, backward_c])
        
    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    #decoder_inputs = Input(shape=(None, num_outputs))
        
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    #decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    #decoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True))
    #decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    
    decoder_inputs = Input(shape=(None, num_outputs), name="decoder_input")
    decoder_lstm = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True, name="decoder_LSTM") #, activation='relu')
    #decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    #bn_decoder = BatchNormalization()(decoder_outputs)
    
    decoder_dense = Dense(num_outputs, activation='softmax', name="decoder_dense")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs, name="model1")
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # summarize layers
    print(model.summary())
    
    # Train model
    encoder_input_data = HDF5Matrix('features.h5', 'training_data')
    decoder_input_data = HDF5Matrix('truth.h5', 'truth')
    decoder_target_data = HDF5Matrix('truthlag.h5', 'truthlag')
    
    #Automicatlly determine batch sizes, train/test splits
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data, shuffle="batch", epochs=2, callbacks=get_callbacks(model))
    for (i, obj) in enumerate(model.get_weights()):
        obj.tofile("weights_layer{0}".format(str(i)))
    
    # Create encoder/decoder model
    encoder_model = Model(encoder_inputs, encoder_states, name="encoder_model")

    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states, name="decoder_model")
    #decoder_model.add(BatchNormalization())
    
    return encoder_model, decoder_model

In [ ]:
def create_lstm_model(num_features, num_outputs=2, latent_dim=100):
    model = Sequential()
    model.add(LSTM(input_shape=(None, None, 20), return_sequences=True))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])



In [23]:
def get_callbacks(model_obj, logging_file="training.log", verbosity=1, early_stopping_patience=None):
    callbacks = list()
    #callbacks.append(ModelCheckpoint(model_obj, save_best_only=True))
    #callbacks.append(CSVLogger(logging_file, append=True))
#    callbacks.append(LossHistory('losshistory.log', append=True))
    callbacks.append(TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='batch'))
#     if learning_rate_epochs:
#         callbacks.append(LearningRateScheduler(partial(step_decay, initial_lrate=initial_learning_rate,
#                                                        drop=learning_rate_drop, epochs_drop=learning_rate_epochs)))
#     else:
#         callbacks.append(ReduceLROnPlateau(factor=learning_rate_drop, patience=learning_rate_patience,
#                                            verbose=verbosity))
#     if early_stopping_patience:
#         callbacks.append(EarlyStopping(verbose=verbosity, patience=early_stopping_patience))
    return callbacks

In [ ]:
#encoder_input_data = HDF5Matrix('features.h5', 'training_data')
training_data.tofile("test.txt")

In [ ]:
# get subset of training data, if needed for faster debugging

#training_data = HDF5Matrix('features.h5', 'training_data')
import h5py
hf = h5py.File('full_dataset_features/features.h5', 'r')
training_data = hf.get('training_data').value
#training_data = training_data[0:128,:,:]
hf.close()

# hf = h5py.File('full_dataset_features/truth.h5', 'r')
# truth = hf.get('truth').value
# truth = truth[0:128,:,:]
# hf.close()

# hf = h5py.File('full_dataset_features/truthlag.h5', 'r')
# truth_lagged = hf.get('truthlag').value
# truth_lagged = truth_lagged[0:128,:,:]
# hf.close()

In [ ]:
training_data.shape
training_data = training_data[:, :, 0:20]
training_data.shape

In [31]:
import tensorflow as tf
import keras
from keras.utils.io_utils import HDF5Matrix
from keras.layers import Input, Dense, Bidirectional, LSTM, Concatenate, CuDNNLSTM, Masking
from keras.models import Model
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard
from keras.layers.normalization import BatchNormalization
from keras.utils import plot_model


# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []

#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))

#encoder_inputs, encoder_states, decoder_inputs, decoder_lstm, decoder_dense, model = train()
#model = train()
encoder_model, decoder_model = create_and_train_model(num_features=24, num_outputs=2, latent_dim=100)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, None, 24)     0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      (None, None, 2)      0                                            
__________________________________________________________________________________________________
encoder_LSTM (CuDNNLSTM)        [(None, 100), (None, 50400       input[0][0]                      
__________________________________________________________________________________________________
decoder_LSTM (CuDNNLSTM)        [(None, None, 100),  41600       decoder_input[0][0]              
                                                                 encoder_LSTM[0][1]               
          

In [ ]:
encoder_model.save_weights("encoder_weights")
decoder_model.save_weights("decoder_weights")

In [ ]:
# load json and create model
json_file = open('model_StS_BiLSTM.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_StS_BiLSTM.h5")
print("Loaded model from disk")

In [37]:
np.fromfile("weights_layer5")

array([ 1.28577856e-14,  3.19465971e-15,  9.97968656e-14,  4.11448434e-17,
        7.68643938e-15,  3.28545861e-20,  1.43525735e-16,  3.25894612e-15,
        6.64276896e-13,  1.20964505e-11,  1.95491603e-24,  2.82222701e-19,
        4.18349289e-12,  2.25621473e-12,  5.15593900e-12, -2.91185474e-20,
        3.34469049e-13, -1.07186738e-16,  9.40922985e-15,  7.12299570e-12,
       -1.06134464e-14,  2.32598686e-16,  3.59302917e-13,  4.71931264e-15,
        3.12599889e-16,  6.31237666e-17,  2.68510950e-12,  5.75789805e-16,
       -4.96457838e-20,  3.12619676e-12,  1.04576860e-10, -2.59939417e-23,
       -2.35683691e-20, -2.15801210e-19, -1.82597776e-16,  9.29035621e-10,
        5.87092109e-13, -2.42708724e-21,  1.18500782e-11,  4.03220301e-19,
        6.26595052e-16,  5.10256625e-11,  4.57002807e-18,  5.64523793e-13,
        5.08412797e-17,  1.53720384e-14,  1.34543882e-14,  5.30674925e-19,
        9.63889773e-15, -1.24110392e-21, -5.37286959e-17, -4.06572748e-22,
       -5.71220422e-19,  

In [ ]:
truth[0,:,:]

In [16]:
def decode_sequence(input_seq, encoder_model, decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    print(states_value)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, 2))
    # Populate the first character of target sequence with the start residue.
    target_seq[0, 0, :] = [0, 1] # decoder tokens one-hot encoding hardcoded here

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_chain = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        print(output_tokens)
        sampled_token_index = (np.argmax(output_tokens[0, -1, :]) + 1) % 2
        decoded_chain.append(sampled_token_index) # flip indices

        # Exit condition: either hit max length
        # or find stop character.
        if len(decoded_chain) >= 100:
            stop_condition = True

        # Update the target sequence (of length 1).
        sampled_residue = [0, 0]
        sampled_residue[sampled_token_index] = 1
        target_seq = np.zeros((1, 1, 2))
        target_seq[0, 0, :] = sampled_residue

        # Update states
        states_value = [h, c]

    return decoded_chain

In [17]:
# To Sean: ran til here
os.getcwd()

'/home/ec2-user/SageMaker/ProteinFragmenter'

In [18]:
#training_data = HDF5Matrix('features.h5', 'training_data')
import h5py
hf = h5py.File('features.h5', 'r')
df_training = hf.get('training_data').value
hf.close()

# import h5py
# hf = h5py.File('truth.h5', 'r')
# df_truth = hf.get('truth').value
# hf.close()

In [12]:
temp = df_training[2:3,:,:]
temp.shape

(1, 100, 24)

In [19]:
result = decode_sequence(temp, encoder_model, decoder_model)

[array([[-4.61753607e-01, -3.99733692e-01, -6.83733076e-02,
         4.30623330e-02, -4.01364863e-01, -5.75511251e-03,
         5.07989749e-02, -4.91346531e-02, -2.35608732e-03,
         1.25614271e-01, -4.13277894e-01,  2.05996484e-01,
        -5.96140146e-01,  4.68936674e-02, -4.25173163e-01,
        -2.83412814e-01,  5.58889151e-01,  1.20824836e-01,
         5.44418514e-01, -7.51006156e-02, -1.09569833e-01,
         4.84094024e-01,  4.51451361e-01, -1.29060462e-01,
        -4.38907176e-01,  6.12171710e-01, -2.31075868e-01,
        -9.26003605e-02, -9.31617245e-02,  4.83249575e-01,
        -9.59240198e-02, -1.62749693e-01, -5.22550717e-02,
         1.24774754e-01, -4.74804193e-01, -6.47274703e-02,
        -5.88638037e-02, -2.35303538e-03,  1.16782390e-01,
         2.42176414e-01, -2.90354133e-01,  1.19337127e-01,
        -2.20247507e-01, -4.46341336e-01, -3.66005637e-02,
        -5.36113679e-01,  3.13704252e-01,  2.12985486e-01,
         2.06312418e-01, -3.54501675e-03, -4.06374276e-

In [14]:
i = 0
for j in result:
    i += j
i

50

In [15]:
result

[0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1]